## Import the relevant packages

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

C:\Users\shrey\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\shrey\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\shrey\AppData\Local\Continuum\anaconda3\envs\py36\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\U

In [2]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


## MNIST - Outline the model

In [3]:
input_size = 784
output_size = 10
hidden_layer_size = 50

In [4]:
# It allows to define several computational graphs and run them independently as different parts of the training
tf.reset_default_graph() 
#It will clears the memory of all variables left from previous runs(reset the computational graphs)

In [5]:
inputs = tf.placeholder(tf.float32, [None, input_size])
targets = tf.placeholder(tf.float32, [None, output_size])

#Declaring variables
weights_1 = tf.get_variable("weights_1", [input_size, hidden_layer_size]) # kXm
biases_1 = tf.get_variable("biases_1", [hidden_layer_size]) # 1Xm
outputs_1 = tf.nn.relu(tf.matmul(inputs,weights_1) + biases_1) #xw+b

weights_2 = tf.get_variable("weights_2", [hidden_layer_size, hidden_layer_size]) 
biases_2 = tf.get_variable("biases_2", [hidden_layer_size])
outputs_2 = tf.nn.relu(tf.matmul(outputs_1,weights_2) + biases_2)

weights_3 = tf.get_variable("weights_3", [hidden_layer_size, output_size])
biases_3 = tf.get_variable("biases_3", [output_size])
outputs = tf.matmul(outputs_2,weights_3) + biases_3

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


## Calculating Loss and Optimizing Algorithm

In [6]:
#tf.nn.softmax_cross_entropy_with_logits() - is a function that applies softmax activation and 
                                            #calculates a cross entropy loss
#Variable Loss 
loss = tf.nn.softmax_cross_entropy_with_logits(logits=outputs, labels=targets)

#tf.reduce_mean() : is a method which finds the mean of the elements of a tensor across a dimension
mean_loss = tf.reduce_mean(loss)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



#### OptimizationMethod

In [7]:
optimize = tf.train.AdamOptimizer(learning_rate=0.001).minimize(mean_loss)

## Calculating the accuracy of the model

In [8]:
#tf.equal() : is a method that checks if two values are equal. In the case of tensor, it does so element wise
out_equal_target = tf.equal(tf.argmax(outputs,1), tf.argmax(targets,1))

#tf.cast(object, data type) : is a method that cast an object to another data type
accuracy = tf.reduce_mean(tf.cast(out_equal_target, tf.float32))

## Batching and Early Stopping

In [9]:
sess = tf.InteractiveSession()
initializer = tf.global_variables_initializer()
sess.run(initializer)

In [10]:
batch_size = 100
batches_number = mnist.train._num_examples // batch_size #batches = #samples/batch_size

#The true early stopping will come if the validation loss starts increasing.
max_epochs = 15
prev_validation_loss = 9999999. #this value large enough to ensure the early stopping wont be triggered on the first epoch.

## MNIST Learning

In [11]:
for epoch_counter in range(max_epochs):
    curr_epoch_loss = 0
    for batch_counter in range(batches_number):
        #Training loss
        input_batch,target_batch = mnist.train.next_batch(batch_size) #will load 100 inputs and 100 targets
        #it optimizes the algorithm and calculates the loss using these exact 100 samples
        _, batch_loss = sess.run([optimize, mean_loss],
                               feed_dict = {inputs:input_batch, targets:target_batch})
        curr_epoch_loss += batch_loss #At the end of each iteration over the batches, we will increase the
                                       #current epoch loss by batch loss.
    curr_epoch_loss /= batches_number
    
    #Validation Loss
    input_batch, target_batch = mnist.validation.next_batch(mnist.validation._num_examples)
    validation_loss, validation_accuracy = sess.run([mean_loss,accuracy], 
                                                    feed_dict={inputs:input_batch, targets:target_batch})
    
    print('Epoch' + str(epoch_counter+1) + 
          '. Training loss:' + '{0:.3f}' .format(curr_epoch_loss) +
          '. Validation loss:' + '{0:.3f}' .format(validation_loss) +
          '. Validation accuracy:' + '{0:.2f}' .format(validation_accuracy * 100) + '%')
    
    #Check when validation loss starts increasing
    if validation_loss > prev_validation_loss:
        break
    prev_validation_loss = validation_loss
    
print('End of training')   

Epoch1. Training loss:0.416. Validation loss:0.195. Validation accuracy:94.48%
Epoch2. Training loss:0.177. Validation loss:0.143. Validation accuracy:95.74%
Epoch3. Training loss:0.134. Validation loss:0.118. Validation accuracy:96.52%
Epoch4. Training loss:0.106. Validation loss:0.104. Validation accuracy:97.08%
Epoch5. Training loss:0.089. Validation loss:0.098. Validation accuracy:97.08%
Epoch6. Training loss:0.075. Validation loss:0.103. Validation accuracy:96.92%
End of training


## Result and Testing

In [12]:
#We'll need to forward propogate and print out the accuracy

#This time the data comes from the test dataset
input_batch, target_batch = mnist.test.next_batch(mnist.test.num_examples)
test_accuracy = sess.run([accuracy], feed_dict={inputs:input_batch, targets:target_batch})
test_accuracy_percent = test_accuracy[0] * 100

print('Test Accuracy:' + '{0:.2f}' . format(test_accuracy_percent) + '%')

Test Accuracy:96.76%
